### CREATING DIMENSION Dim_Model

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
drop table if exists cars_catalog.gold.dim_model 

In [0]:
df_src= spark.sql('''
select  distinct(model_id),model_category 
from parquet.`abfss://silver@saanushe2e.dfs.core.windows.net/carsales`''')
df_src.display()


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
%python
if not spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    df_sink =spark.sql(''' select 1 as dim_model_key,(model_id),model_category 
    from parquet.`abfss://silver@saanushe2e.dfs.core.windows.net/carsales`
    where 1 =0 ''')

df_filter = df_src.join(df_sink,df_src.model_id==df_sink.model_id,'left').select(df_src.model_id,df_src.model_category,coalesce(df_sink.dim_model_key).alias('dim_model_key'))



In [0]:
dbutils.widgets.text('incremental_flag','0')
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)


0


In [0]:
df_filter_old = df_filter.filter(col('dim_model_key').isNotNull())
df_filter_new = df_filter.filter(col('dim_model_key').isNull())



In [0]:
if (incremental_flag == '0'):
  max_value = 1
else:
  max_value_df = spark.sql('''
  select max(dim_branch_key) as max_value
  from gold.dim_model''')
  max_value = max_value_df.collect()[0][0]

  df_filter_new =df_filter_new.withColumn('dim_model_key',max_value+monotonically_increasing_id())
 

  

In [0]:
df_final  = df_filter_new.union(df_filter_old)

In [0]:
# Incremental Run

if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
   delta_tbl = DeltaTable.forPath(spark,'abfss://gold@saanushe2e.dfs.core.windows.net/dim_model')
   delta_tbl.alias("trg").merge(df_final.alias("src"),"trg.dim_model_key = src.dim_model_key")\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll().execute()
else:
  df_final.write.format("delta")\
                .mode ("overwrite")\
                .option("path","abfss://gold@saanushe2e.dfs.core.windows.net/dim_model")\
                .saveAsTable("cars_catalog.gold.dim_model")
